<a href="https://colab.research.google.com/github/art-lmk/House-Price-Regressor-app/blob/main/House_Price_Predictor_app_using_Streamlit_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install streamlit -q
!pip install pyngrok -q
!pip install joblib -q
!pip install scikit-learn -q

print ('Successfully')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.3 MB/s eta 0:00:00
Successfully


In [4]:
# Cell 2: Train the model and save it
import pandas as pd
# from sklearn.datasets import fetch_california_housing # Using California Housing as Boston is deprecated
from sklearn.linear_model import LinearRegression
import joblib

# Load dataset
# california = fetch_california_housing()
# df = pd.DataFrame(california.data, columns=california.feature_names)
df = pd.read_csv('housing.csv')
# df['MEDV'] = california.target * 100000 # Convert to approximate dollar value


# Define features and target
# Using: MedInc (Median Income), HouseAge, AveRooms (Average Rooms)
feature_names = ['RM', 'LSTAT', 'PTRATIO']
X = df[feature_names]
y = df['MEDV']

# Train the model
model = LinearRegression()
model.fit(X, y)

# Save the model
joblib.dump(model, 'house_price_model.pkl')
print("Model trained and saved! Features used:", feature_names)
print(f"Model Intercept: ${model.intercept_:.2f}")
for i, feature in enumerate(feature_names):
    print(f" Coefficient for {feature}: {model.coef_[i]:.2f}")

Model trained and saved! Features used: ['RM', 'LSTAT', 'PTRATIO']
Model Intercept: $415464.40
 Coefficient for RM: 86565.24
 Coefficient for LSTAT: -10849.34
 Coefficient for PTRATIO: -19492.12


In [5]:
# Cell 3: Write the Streamlit app code to a file (This is the frontend)
%%writefile app.py
import streamlit as st
import joblib
import numpy as np

# Load the trained model
model = joblib.load('house_price_model.pkl')

# App UI
st.set_page_config(page_title="🏠 House Price Predictor", layout="centered")
st.title("🏠 AI House Price Predictor")
st.markdown("""
This app predicts **California House Prices** based on:
- **pupil-teacher ratio per town** of the household
- **Lower status population** of the house
- **Average Number of Rooms**
""")

# Create input fields with sliders
st.header("Enter House Details")
ptratio = st.slider('Pupil-Teacher Ratio (PTRATIO)', 12.0, 23.0, 19.0, 0.1)
lstat = st.slider('% Lower Status Population (LSTAT)', 1.0, 40.0, 12.0, 0.5)
ave_rooms = st.slider('Average Number of Rooms', 1.0, 10.0, 5.0, 0.1)

# Format the input for the model
input_data = np.array([[ptratio, lstat, ave_rooms]])

# Make prediction
if st.button('Predict Price', type='primary'):
    prediction = model.predict(input_data)[0]
    st.balloons()
    st.success(f'### 🎯 Predicted House Price: **${prediction:,.2f}**')
    st.info("💡 This is an estimate based on a machine learning model.")

# Add some explanation
with st.expander("ℹ️ How does this work?"):
    st.markdown("""
    This is a **Multiple Linear Regression** model. It learned the relationship between house features and their prices from a large dataset of California homes.
    - It was built using `scikit-learn` and `Streamlit`.
    - The model is running live right here in the cloud on Google Colab!
    """)

Writing app.py


In [12]:
# Cell 4: Using Environment Variables
import os
from google.colab import userdata

# Install required packages
!pip install pyngrok -q
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz -O ngrok.tgz
!tar -xzf ngrok.tgz

# Use Google Colab's built-in secrets manager (most secure)
# First, set your token in Colab secrets: click on the key icon in the left sidebar
try:
    ngrok_token = userdata.get('NGROK_TOKEN')
    !./ngrok authtoken $ngrok_token
    print("✅ Ngrok authenticated using Colab secrets!")
except userdata.SecretNotFoundError:
    print("Please set NGROK_TOKEN in Colab secrets (left sidebar key icon)")

--2025-09-01 08:28:44--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 75.2.60.68, 13.248.244.96, 99.83.220.108, ...
Connecting to bin.equinox.io (bin.equinox.io)|75.2.60.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9279009 (8.8M) [application/octet-stream]
Saving to: ‘ngrok.tgz’

ngrok.tgz           100%[===================>]   8.85M  35.8MB/s    in 0.2s    

2025-09-01 08:28:44 (35.8 MB/s) - ‘ngrok.tgz’ saved [9279009/9279009]

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Ngrok authenticated using Colab secrets!


In [13]:
# Cell 5: Launch the Streamlit app with ngrok
from pyngrok import ngrok
import time
import threading

# Function to run the Streamlit server in the background
def run_streamlit():
    # This command runs Streamlit with settings that work well in Colab
    !streamlit run app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false

# Terminate any existing ngrok tunnels to avoid conflicts
# ngrok.kill() # Removed this line as it is not necessary and might interfere with authentication

# Set up a secure tunnel to Streamlit's port (8501)
print("🚀 Starting ngrok tunnel...")
public_url = ngrok.connect(addr='8501', proto='http')
print(f"✅ Tunnel created successfully!")

# Print the public URL - THIS IS THE LINK FOR YOUR PHONE
print("✨ Your AI House Price Predictor is now live!")
print("🌐 ** Public App URL: **")
print(f"   {public_url}")
print("\n📱 Open this URL on your phone's browser to use the app!")

# Start the Streamlit server in a separate thread
# This allows the ngrok tunnel to stay active while Streamlit runs
print("\n⏳ Starting the AI server... (please wait 5-10 seconds)")
thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

# Give the server some time to start up completely
time.sleep(8)

# Final instructions
print("--------------------------------------------------------------------------------")
print("🎯 App is ready! Copy the URL above and open it on any device.")
print("⚠️  Keep this Colab tab running to keep your app online.")
print("   If you close this tab, your app will shut down.")
print("--------------------------------------------------------------------------------")

🚀 Starting ngrok tunnel...
✅ Tunnel created successfully!
✨ Your AI House Price Predictor is now live!
🌐 ** Public App URL: **
   NgrokTunnel: "https://af06480766b8.ngrok-free.app" -> "http://localhost:8501"

📱 Open this URL on your phone's browser to use the app!

⏳ Starting the AI server... (please wait 5-10 seconds)



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.23.45.228:8501

--------------------------------------------------------------------------------
🎯 App is ready! Copy the URL above and open it on any device.
⚠️  Keep this Colab tab running to keep your app online.
   If you close this tab, your app will shut down.
--------------------------------------------------------------------------------


### Troubleshooting ngrok Authentication Error (ERR_NGROK_4018)

This error occurs when ngrok cannot authenticate, usually because the authtoken is missing or incorrect.

**To fix this:**

1.  Open the **Secrets** panel in the left sidebar (click the 🔑 icon).
2.  Click **New secret**.
3.  Set the **Name** to `NGROK_TOKEN`.
4.  Set the **Value** to your actual ngrok authtoken (found on your ngrok dashboard).
5.  Ensure **Notebook access** is toggled ON for this secret.
6.  Re-run the code cell that uses `userdata.get('NGROK_TOKEN')` (Cell 4) and then the cell that launches ngrok (Cell 5).